In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

from tqdm import tqdm
import time
import json

# multiple output from cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
with open("Data/ETFdata_etfdb_raw.json", "r") as openfile:
    # Reading from json file
    data = json.load(openfile)

field_dfs = {field:pd.DataFrame(info) for field,info in data.items()}

In [13]:
# check for dublicates
field_dfs["overview"][field_dfs["overview"].duplicated(subset = ["Symbol"], keep = False)]

,Symbol,ETF Name,Asset Class New,Total Assets ($MM),YTD Price Change,Avg. Daily Share Volume (3mo),Previous Closing Price,ETF Database Pro
2406,JHPI,John Hancock Preferred Income ETF,Bond,$14,-9.38%,8765,$22.01,NaN
2407,JHPI,John Hancock Preferred Income ETF,Preferred Stock,$14,-9.38%,8765,$22.01,NaN


In [14]:
# drop dublicates (keep last)
for field, df in field_dfs.items():
    field_dfs[field] = df.drop_duplicates(subset=["Symbol"], keep="last").reset_index(drop = True)# .set_index("Symbol")

In [15]:
# # look at all dfs
# for field, df in field_dfs.items():
#     df.head()
#     print(f"df field: {field}")

In [16]:
# # get columns for all fields
# for field, df in field_dfs.items():
#     print(f"'{field}':[")
#     for col in df.columns:
#         print(f"\t'{col}',")
#     print(f"],")

In [17]:
columns_toget ={
    'overview':[
        'Symbol',
        'ETF Name',
        'Asset Class New',
        'Total Assets ($MM)',
#         'YTD Price Change',
        'Avg. Daily Share Volume (3mo)',
#         'Previous Closing Price',
#         'ETF Database Pro',
    ],
    'returns':[
        'Symbol',
#         'ETF Name',
#         '1 Week',
#         '1 Month',
#         'YTD',
#         '1 Year',
#         '3 Year',
#         '5 Year',
#         'Returns Rating',
    ],
    'expenses':[
        'Symbol',
#         'ETF Name',
#         'Asset Class New',
#         'Total Assets ($MM)',
        'Expense Ratio',
#         'ETF Database Pro',
    ],
    'esg':[
        'Symbol',
#         'ETF Name',
        'ESG Score',
        'ESG Score Peer Percentile (%)',
        'ESG Score Global Percentile (%)',
#         'Carbon Intensity (Tons of CO2e / $M Sales)',
#         'SRI Exclusion Criteria (%)',
#         'Sustainable Impact Solutions (%)',
    ],
    'dividends':[
        'Symbol',
#         'ETF Name',
#         'Annual Dividend Rate',
#         'Dividend Date',
#         'Dividend Frequency New',
#         'Annual Dividend Yield',
#         'Dividend Rating',
    ],
    'risk':[
        'Symbol',
#         'ETF Name',
#         'Standard Deviation',
        'P/E Ratio',
        'Beta',
#         '5-Day Volatility',
#         '20-Day Volatility',
#         '50-Day Volatility',
#         '200-Day Volatility',
    ],
    'holdings':[
        'Symbol',
#         'ETF Name',
        'Issuer New',
        '# of Holdings',
        '% In Top 10',
#         '% In Top 15 New',
#         '% In Top 50 New',
#         'Complete',
#         'Concentration Rating',
    ],
    'realtime-ratings':[
        'Symbol',
#         'ETF Name',
        'Liquidity',
        'Expenses',
#         'Returns',
#         'Volatility',
#         'Overall',
#         'Dividend',
#         'Concentration',
    ],
}

In [18]:
# filter dfs by columns to use and merge to one
init = True
for field, df in field_dfs.items():
    if init:
        df_allfields = df[columns_toget[field]]
#         df_allfields.head()
        init = False
    else:
        df_allfields = df_allfields.merge(df[columns_toget[field]], on = "Symbol", how='left')
#         df_allfields.head()

In [19]:
# df_allfields.info()
# df_allfields.iloc[0,:]

In [20]:
# print("fix_colnames = [")
# for col in df_allfields.columns:
#     print(f"\t'{col}',")
# print("]")

In [21]:
# helper funcions
def percent2float(p):
    return float(str(p).strip('%'))/100  

def dollar2float(d):
    return float(str(d).strip('$').replace(",", ""))


# cols to fix
percent_cols = [
    "Expense Ratio",
    "ESG Score Peer Percentile (%)",
    "ESG Score Global Percentile (%)",
    "% In Top 10", 
]

dollar_cols = [
    "Total Assets ($MM)",
]

float_cols = [
    "Avg. Daily Share Volume (3mo)", 
    "ESG Score",
    "P/E Ratio",                      
    "Beta",                                             
]

int_cols = [
    "# of Holdings",
]

# fix col values
df_allfields[percent_cols] = df_allfields[percent_cols].apply(lambda x: [percent2float(val) for val in x])
df_allfields[dollar_cols] = df_allfields[dollar_cols].apply(lambda x: [dollar2float(val) for val in x])
df_allfields[float_cols] = df_allfields[float_cols].apply(lambda x: x.astype(float))
df_allfields[int_cols] = df_allfields[int_cols].apply(lambda x: x.astype(float))

# new col names
colnames_fix = [
    'Symbol',
    'ETF Name',
    'Asset Class',
    'Total Assets ($MM)',
    'Avg. Daily Share Volume (3mo)',
    'Expense Ratio',
    'ESG Score',
    'ESG Score Peer Percentile',
    'ESG Score Global Percentile',
    'P/E Ratio',
    'Beta',
    'Issuer',
    '# of Holdings',
    '% In Top 10',
    'Liquidity',
    'Expenses',
]

# fix col names
df_allfields.columns = colnames_fix

In [22]:
df_allfields.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2994 entries, 0 to 2993
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Symbol                         2994 non-null   object 
 1   ETF Name                       2994 non-null   object 
 2   Asset Class                    2994 non-null   object 
 3   Total Assets ($MM)             2986 non-null   float64
 4   Avg. Daily Share Volume (3mo)  2878 non-null   float64
 5   Expense Ratio                  2994 non-null   float64
 6   ESG Score                      2233 non-null   float64
 7   ESG Score Peer Percentile      2233 non-null   float64
 8   ESG Score Global Percentile    2233 non-null   float64
 9   P/E Ratio                      1539 non-null   float64
 10  Beta                           2436 non-null   float64
 11  Issuer                         2994 non-null   object 
 12  # of Holdings                  2975 non-null   f

In [23]:
# field_dicts = {}

# for field, df in field_dfs.items():
#     field_dict = df.to_dict(orient="index")
#     field_dicts.update({field:field_dict})

# etf_dict = {etf:{} for etf in list(field_dicts.values())[0].keys()}

# for field, fdict in field_dicts.items():
#     for etf, info in fdict.items():
#         etf_dict[etf].update(info)

# list(etf_dict.get("SPY").keys())

In [24]:
# crate dict where symbols are keys
dict_allfields = df_allfields.set_index("Symbol").to_dict(orient="index")

In [25]:
# write out to files

with open("Data/ETFdata_etfdb_clean.json", "w") as outfile:
    # Write to json file
    json.dump(dict_allfields, outfile)
    
df_allfields.to_csv("Data/ETFdata_etfdb_clean.csv", index=False)
